# DATASHEET DRILL — 2N3904 Datasheet Walkthrough
## Reading Every Section of a Real BJT Datasheet

**Module 03 — Transistors (BJT) | Notebook 03**

---

The datasheet is the transistor's contract with you. Everything you need to know about how
a device will behave — and the limits you must never exceed — is in that document.

In this notebook we walk through the **2N3904 NPN transistor datasheet** section by section.
By the end, you will be able to pick up any BJT datasheet and quickly extract the information
needed for your circuit design.

**What you will learn:**
- How to read absolute maximum ratings (and why you should NEVER hit them)
- How to interpret h_FE ranges and test conditions
- How to use characteristic curves for design
- The Safe Operating Area (SOA) — the box you must stay inside
- Thermal derating — how much power you can ACTUALLY dissipate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import ipywidgets as widgets
from IPython.display import display, HTML

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

---
## Section 1: Part Overview and Pin Identification

### 2N3904 — NPN General-Purpose Transistor

- **Type**: NPN silicon transistor
- **Package**: TO-92 (small plastic through-hole)
- **Typical applications**: Small-signal amplification, switching
- **Complementary PNP**: 2N3906

### Pin Identification — CAUTION!

```
    TO-92 Package (flat side facing you, leads pointing down)
    
    ON Semiconductor / Fairchild:        Some other manufacturers:
    
        ___________                          ___________
       /           \                        /           \
      |   (front)   |                      |   (front)   |
      |             |                      |             |
       \___________/                        \___________/
        |    |    |                          |    |    |
        E    B    C                          C    B    E
```

**CRITICAL**: Pin order is NOT standardized across manufacturers!
- ON Semiconductor / Fairchild / most: **E-B-C** (left to right)
- Some Asian manufacturers: **C-B-E** or other orders

**ALWAYS check the datasheet for YOUR specific part.** If you get the pins wrong, the
transistor will not work (and may be damaged). A diode test with your Klein MM300 can
confirm pin identity (see Notebook 00).

---
## Section 2: Absolute Maximum Ratings

These are the **never-exceed** limits. Going beyond ANY of these, even briefly, risks
permanent damage.

| Parameter | Symbol | Value | What It Means |
|-----------|--------|-------|---------------|
| Collector-Emitter Voltage | V_CEO | 40V | Max voltage across C-E with base open |
| Collector-Base Voltage | V_CBO | 60V | Max voltage across C-B with emitter open |
| Emitter-Base Voltage | V_EBO | 6V | Max reverse voltage on B-E junction |
| Collector Current | I_C | 200 mA | Max continuous collector current |
| Total Power Dissipation | P_D | 625 mW | Max power at 25 C ambient |
| Junction Temperature | T_J | 150 C | Absolute max silicon temperature |

### What Each Rating Means in Practice

**V_CEO = 40V**: If your circuit has a 12V supply, you have plenty of margin. But if you
are switching an inductive load without a flyback diode, the voltage spike can easily
exceed 40V and destroy the transistor.

**V_EBO = 6V**: This one catches beginners. The base-emitter junction can only tolerate
**6V in reverse**. If you apply a negative voltage to the base to turn it off fast (common
in switching circuits), keep it under 6V.

**I_C = 200 mA**: This is the max continuous current. You can briefly exceed this (pulsed),
but sustained current above 200 mA will overheat the bond wires.

**P_D = 625 mW**: This is only valid at **25 C ambient**. At higher temperatures, you get
LESS. We will calculate exactly how much less below.

---
## Section 3: Electrical Characteristics — h_FE (Beta)

This is the most frequently referenced section. Here is what the datasheet actually says:

| Parameter | Test Conditions | Min | Typ | Max | Unit |
|-----------|----------------|-----|-----|-----|------|
| h_FE | I_C = 0.1 mA, V_CE = 1V | 40 | — | — | — |
| h_FE | I_C = 1 mA, V_CE = 1V | 70 | — | — | — |
| h_FE | I_C = 10 mA, V_CE = 1V | 100 | — | 300 | — |
| h_FE | I_C = 50 mA, V_CE = 1V | 60 | — | — | — |
| h_FE | I_C = 100 mA, V_CE = 1V | 30 | — | — | — |

### Critical Observations

1. **Beta is NOT constant** — it varies with collector current! Peak beta is around 10-50 mA.
   At very low current (0.1 mA) and very high current (100 mA), beta drops significantly.

2. **The min-max range is huge**: 100 to 300 at I_C = 10 mA. You might get a part with
   beta = 120 or one with beta = 280. Your circuit must work with ANY value in this range.

3. **Test conditions matter**: h_FE is specified at V_CE = 1V. At very low V_CE (near
   saturation), effective beta drops further.

4. **At 100 mA, min beta is only 30!** This is far from the "beta = 200" often assumed
   in back-of-envelope calculations.

In [ ]:
# Recreate hFE vs IC curve from datasheet data

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Datasheet data points for hFE vs IC
I_C_data = np.array([0.1, 1, 10, 50, 100])  # mA
hFE_min = np.array([40, 70, 100, 60, 30])     # minimum beta
hFE_typ = np.array([80, 150, 200, 150, 80])   # typical (interpolated)
hFE_max = np.array([120, 250, 300, 250, 150])  # maximum (estimated from typical spread)

# Create smooth interpolated curves
I_C_smooth = np.logspace(-1, 2.3, 200)  # 0.1 mA to 200 mA

# Simple interpolation using log scale for I_C
from scipy import interpolate
f_min = interpolate.interp1d(np.log10(I_C_data), hFE_min, kind='quadratic', fill_value='extrapolate')
f_typ = interpolate.interp1d(np.log10(I_C_data), hFE_typ, kind='quadratic', fill_value='extrapolate')
f_max = interpolate.interp1d(np.log10(I_C_data), hFE_max, kind='quadratic', fill_value='extrapolate')

hFE_min_smooth = np.maximum(f_min(np.log10(I_C_smooth)), 10)
hFE_typ_smooth = np.maximum(f_typ(np.log10(I_C_smooth)), 20)
hFE_max_smooth = np.maximum(f_max(np.log10(I_C_smooth)), 30)

# Left plot: hFE vs IC (log scale for IC)
ax1.semilogx(I_C_smooth, hFE_typ_smooth, 'b-', linewidth=2.5, label='Typical')
ax1.fill_between(I_C_smooth, hFE_min_smooth, hFE_max_smooth, alpha=0.2, color='blue',
                 label='Min-Max range')
ax1.semilogx(I_C_smooth, hFE_min_smooth, 'b--', linewidth=1, alpha=0.7, label='Minimum')
ax1.semilogx(I_C_smooth, hFE_max_smooth, 'b--', linewidth=1, alpha=0.7, label='Maximum')

# Mark datasheet points
ax1.plot(I_C_data, hFE_min, 'rv', markersize=10, label='Datasheet min values')
ax1.plot(I_C_data, hFE_typ, 'gs', markersize=10, label='Typical values')

# Annotation
ax1.annotate('Peak beta\n~10-50 mA', xy=(20, 210), xytext=(50, 280),
             fontsize=11, fontweight='bold', color='green',
             arrowprops=dict(arrowstyle='->', color='green', lw=2),
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax1.annotate('Beta drops at\nhigh current!', xy=(100, 80), xytext=(30, 50),
             fontsize=11, fontweight='bold', color='red',
             arrowprops=dict(arrowstyle='->', color='red', lw=2),
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax1.set_xlabel('Collector Current I_C (mA)', fontsize=12)
ax1.set_ylabel('h_FE (Current Gain)', fontsize=12)
ax1.set_title('2N3904: h_FE vs I_C\n(from datasheet)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9, loc='upper left')
ax1.set_xlim(0.1, 200)
ax1.set_ylim(0, 350)

# Right plot: Why this matters for design
# Show what I_C you actually get for I_B = 50 uA
I_B_design = 0.050  # 50 uA in mA
I_C_actual_min = hFE_min_smooth * I_B_design  # mA
I_C_actual_max = hFE_max_smooth * I_B_design  # mA

ax2.semilogx(I_C_smooth, I_C_smooth, 'k-', linewidth=1, alpha=0.3, label='I_C = I_C (reference)')
ax2.fill_between(I_C_smooth, I_C_actual_min, I_C_actual_max, alpha=0.3, color='orange')

# For a specific operating point
ax2.axhline(y=10, color='green', linestyle='--', linewidth=2, label='Target I_C = 10 mA')

# Find the range of I_B needed for I_C = 10 mA
I_B_for_10mA_max_beta = 10 / 300  # mA -> 33 uA
I_B_for_10mA_min_beta = 10 / 100  # mA -> 100 uA

ax2.annotate(f'For I_C = 10 mA:\nI_B = {I_B_for_10mA_max_beta*1e3:.0f}-{I_B_for_10mA_min_beta*1e3:.0f} uA\n(3x range!)',
             xy=(10, 10), xytext=(0.3, 12),
             fontsize=11, fontweight='bold',
             arrowprops=dict(arrowstyle='->', color='black', lw=2),
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax2.set_xlabel('Collector Current I_C (mA)', fontsize=12)
ax2.set_ylabel('Expected I_C (mA) for fixed I_B', fontsize=12)
ax2.set_title('Why Beta Variation Matters\n(Same I_B gives WIDE range of I_C)', 
              fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.set_xlim(0.1, 200)
ax2.set_ylim(0, 20)

plt.tight_layout()
plt.show()

print("Design rule: NEVER rely on a specific beta value.")
print("For switches: overdrive the base (forced beta = 10).")
print("For amplifiers: use negative feedback (R_E) to make gain independent of beta.")

---
## Section 4: Saturation Voltages

The datasheet specifies V_CE(sat) and V_BE(sat) under specific test conditions:

| Parameter | Condition | Min | Typ | Max | Unit |
|-----------|-----------|-----|-----|-----|------|
| V_CE(sat) | I_C = 10 mA, I_B = 1 mA | — | 0.1 | 0.2 | V |
| V_CE(sat) | I_C = 50 mA, I_B = 5 mA | — | 0.2 | 0.3 | V |
| V_BE(sat) | I_C = 10 mA, I_B = 1 mA | — | 0.65 | 0.85 | V |
| V_BE(sat) | I_C = 50 mA, I_B = 5 mA | — | 0.75 | 0.95 | V |

### Key Observations

1. **V_CE(sat) increases with current**: 0.1V at 10 mA, 0.2V at 50 mA. The transistor's
   internal resistance causes a larger voltage drop at higher currents.

2. **Test conditions specify forced beta = 10**: I_C/I_B = 10 in both cases. This confirms
   the "overdrive base by 10x" design rule we learned earlier.

3. **V_BE varies with current**: It is NOT exactly 0.7V. It ranges from 0.65 to 0.95V
   depending on current. For design, use 0.7V as a first approximation, then refine.

4. **Both have a MAX column**: For worst-case design, use the MAX values.

---
## Section 5: Switching Characteristics and Frequency Response

| Parameter | Symbol | Value | What It Means |
|-----------|--------|-------|---------------|
| Transition Frequency | f_T | 300 MHz | Frequency where current gain drops to 1 |
| Output Capacitance | C_ob | 4.0 pF | Capacitance of CB junction (affects speed) |
| Delay Time | t_d | 35 ns | Time from base signal to collector response |
| Rise Time | t_r | 35 ns | Time for collector current to rise 10-90% |
| Storage Time | t_s | 200 ns | Time to clear stored charge from base |
| Fall Time | t_f | 50 ns | Time for collector current to fall 90-10% |

### What This Means in Practice

**f_T = 300 MHz**: The 2N3904 has useful gain up to tens of MHz. For audio frequencies
(20 Hz - 20 kHz), it has plenty of bandwidth.

**Storage time = 200 ns**: This is the slowest part of the switching cycle. When saturated,
the base is full of stored charge that must be removed before the transistor can turn off.
For switching at >1 MHz, this becomes the limiting factor.

**Total switching time**: t_on = t_d + t_r = 70 ns. t_off = t_s + t_f = 250 ns.
Maximum practical switching frequency ~ 1 / (t_on + t_off) ~ 3 MHz.

In [ ]:
# Plot Safe Operating Area (SOA)

fig, ax = plt.subplots(figsize=(10, 7))

# SOA boundaries for 2N3904
V_CEO_max = 40      # V
I_C_max = 0.200     # A (200 mA)
P_D_max = 0.625     # W (625 mW)

# Create SOA region
V_CE = np.linspace(0.01, V_CEO_max, 1000)

# Current limit
I_C_current_limit = np.full_like(V_CE, I_C_max)

# Power limit: P = V_CE * I_C => I_C = P / V_CE
I_C_power_limit = P_D_max / V_CE

# The SOA boundary is the MINIMUM of all limits
I_C_SOA = np.minimum(I_C_current_limit, I_C_power_limit)

# Voltage limit
I_C_SOA[V_CE > V_CEO_max] = 0

# Plot on log-log scale (traditional SOA presentation)
ax.loglog(V_CE, I_C_SOA * 1e3, 'r-', linewidth=3, label='SOA boundary')
ax.fill_between(V_CE, 0.01, I_C_SOA * 1e3, alpha=0.15, color='green', label='Safe Operating Area')

# Mark the individual limits
ax.axhline(y=200, color='blue', linestyle='--', linewidth=1.5, alpha=0.7, label=f'I_C max = {I_C_max*1e3:.0f} mA')
ax.axvline(x=40, color='purple', linestyle='--', linewidth=1.5, alpha=0.7, label=f'V_CEO max = {V_CEO_max} V')

# Power limit hyperbola
V_CE_power = np.linspace(3.125, 40, 100)  # where power limit < current limit
I_C_power = P_D_max / V_CE_power
ax.loglog(V_CE_power, I_C_power * 1e3, 'orange', linewidth=1.5, linestyle=':', 
          label=f'P_D max = {P_D_max*1e3:.0f} mW')

# Plot some example operating points
examples = [
    (0.2, 15, 'Saturated switch\n(safe)', 'green'),
    (5, 10, 'Active amplifier\n(safe)', 'green'),
    (2, 100, 'High current\n(safe)', 'green'),
    (20, 50, 'P = 1W!\n(UNSAFE!)', 'red'),
    (10, 100, 'P = 1W!\n(UNSAFE!)', 'red'),
]

for v_ce, i_c, label, color in examples:
    p = v_ce * i_c / 1000  # watts
    marker = 'o' if color == 'green' else 'x'
    ax.plot(v_ce, i_c, marker, color=color, markersize=12, markeredgewidth=3, zorder=5)
    ax.annotate(label, xy=(v_ce, i_c), xytext=(v_ce * 1.5, i_c * 1.3),
                fontsize=9, color=color, fontweight='bold',
                arrowprops=dict(arrowstyle='->', color=color, lw=1.5))

ax.set_xlabel('V_CE (V)', fontsize=13)
ax.set_ylabel('I_C (mA)', fontsize=13)
ax.set_title('2N3904 Safe Operating Area (SOA)\nStay inside the green region!', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=9, loc='lower left')
ax.set_xlim(0.1, 50)
ax.set_ylim(0.1, 300)

plt.tight_layout()
plt.show()

print("The SOA is bounded by THREE limits:")
print(f"  1. Maximum current:  I_C < {I_C_max*1e3:.0f} mA (horizontal line)")
print(f"  2. Maximum voltage:  V_CE < {V_CEO_max} V (vertical line)")
print(f"  3. Maximum power:    P = V_CE * I_C < {P_D_max*1e3:.0f} mW (hyperbola)")
print(f"\nThe power limit is the most commonly violated — it's easy to forget!")

---
## The Material Science Why — Thermal Limits

Every limit in the datasheet traces back to what happens in the silicon:

### Why V_CEO = 40V?
At high reverse voltage, the depletion region at the collector-base junction widens. At
40V, it gets wide enough for **avalanche breakdown** — carriers gain enough energy to knock
other electrons free, creating a runaway current. The junction conducts uncontrollably.

### Why V_EBO = only 6V?
The emitter-base junction is designed to be forward-biased. The emitter is heavily doped
(N+), making the depletion region very narrow. A narrow depletion region breaks down at a
LOWER voltage than a wide one. This is why V_EBO is so much lower than V_CEO.

### Why P_D = 625 mW?
Power dissipation heats the silicon. The junction temperature cannot exceed 150 C or the
silicon starts behaving unpredictably — intrinsic carrier concentration rises, leakage
increases, and the transistor can enter **thermal runaway**: heat increases leakage, which
increases power, which increases heat, until the device is destroyed.

The 625 mW rating assumes a thermal resistance of 200 C/W from junction to ambient air
in the TO-92 package:

$$P_{max} = \frac{T_{J(max)} - T_{ambient}}{\theta_{JA}} = \frac{150°C - 25°C}{200°C/W} = 0.625W$$

In [ ]:
# Thermal derating calculation

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 2N3904 thermal parameters
T_J_max = 150      # C
theta_JA = 200     # C/W (junction to ambient, TO-92)

T_ambient = np.linspace(-40, 150, 200)  # C

# Maximum power at each ambient temperature
P_max = np.maximum((T_J_max - T_ambient) / theta_JA, 0)  # watts

# Left: Thermal derating curve
ax1.plot(T_ambient, P_max * 1e3, 'r-', linewidth=2.5)
ax1.fill_between(T_ambient, 0, P_max * 1e3, alpha=0.15, color='green', label='Allowed power')

# Mark key points
ax1.plot(25, 625, 'ko', markersize=10, zorder=5)
ax1.annotate('625 mW @ 25C\n(datasheet spec)', xy=(25, 625), xytext=(60, 700),
             fontsize=11, arrowprops=dict(arrowstyle='->', lw=2),
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax1.plot(50, (150-50)/200*1000, 'bo', markersize=10, zorder=5)
ax1.annotate(f'{(150-50)/200*1000:.0f} mW @ 50C', xy=(50, (150-50)/200*1000), xytext=(80, 400),
             fontsize=11, arrowprops=dict(arrowstyle='->', lw=2, color='blue'), color='blue',
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax1.plot(85, (150-85)/200*1000, 'ro', markersize=10, zorder=5)
ax1.annotate(f'{(150-85)/200*1000:.0f} mW @ 85C\n(inside a hot enclosure)', 
             xy=(85, (150-85)/200*1000), xytext=(95, 200),
             fontsize=10, arrowprops=dict(arrowstyle='->', lw=2, color='red'), color='red',
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax1.set_xlabel('Ambient Temperature (C)', fontsize=12)
ax1.set_ylabel('Maximum Power Dissipation (mW)', fontsize=12)
ax1.set_title('2N3904 Thermal Derating Curve\n(TO-92 package, free air)', 
              fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.set_xlim(-40, 160)
ax1.set_ylim(0, 1000)

# Right: Junction temperature for a given power dissipation
P_diss = np.linspace(0, 0.8, 200)  # watts
T_ambient_values = [25, 40, 60, 85]
colors = ['#2ecc71', '#3498db', '#e67e22', '#e74c3c']

for T_a, color in zip(T_ambient_values, colors):
    T_J = T_a + P_diss * theta_JA
    ax2.plot(P_diss * 1e3, T_J, color=color, linewidth=2.5, label=f'T_ambient = {T_a}C')

ax2.axhline(y=150, color='red', linestyle='--', linewidth=2, label='T_J max = 150C')
ax2.fill_between(P_diss * 1e3, 150, 200, alpha=0.15, color='red')
ax2.text(400, 165, 'DANGER ZONE', fontsize=14, fontweight='bold', color='red', ha='center')

ax2.set_xlabel('Power Dissipation (mW)', fontsize=12)
ax2.set_ylabel('Junction Temperature (C)', fontsize=12)
ax2.set_title('Junction Temperature vs Power\n(Higher ambient = less headroom)', 
              fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.set_xlim(0, 800)
ax2.set_ylim(0, 200)

plt.tight_layout()
plt.show()

print("Practical implication:")
print(f"  At 25C ambient: max {625} mW")
print(f"  At 50C ambient: max {int((150-50)/200*1000)} mW (20% less!)")
print(f"  At 85C ambient: max {int((150-85)/200*1000)} mW (HALF the datasheet value!)")
print(f"\nIf your circuit is inside an enclosure that gets warm, you may have far less")
print(f"power budget than the front-page 625 mW suggests.")

---
## Section 6: Characteristic Curves — V_CE(sat) vs I_C

The datasheet includes graphs showing how V_CE(sat) varies with collector current.
This is important for calculating power dissipation in switch circuits.

In [ ]:
# V_CE(sat) vs I_C (recreated from datasheet curves)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# V_CE(sat) vs I_C at forced beta = 10
I_C_sat = np.array([1, 5, 10, 20, 50, 100, 150, 200])  # mA
V_CE_sat_typ = np.array([0.04, 0.06, 0.09, 0.12, 0.18, 0.25, 0.35, 0.50])  # V (typical)
V_CE_sat_max = np.array([0.08, 0.12, 0.18, 0.22, 0.30, 0.40, 0.55, 0.75])  # V (max)

ax1.semilogy(I_C_sat, V_CE_sat_typ * 1e3, 'b-o', linewidth=2, markersize=8, label='Typical')
ax1.semilogy(I_C_sat, V_CE_sat_max * 1e3, 'r--s', linewidth=2, markersize=8, label='Maximum')
ax1.fill_between(I_C_sat, V_CE_sat_typ * 1e3, V_CE_sat_max * 1e3, alpha=0.2, color='orange')

ax1.set_xlabel('Collector Current I_C (mA)', fontsize=12)
ax1.set_ylabel('V_CE(sat) (mV)', fontsize=12)
ax1.set_title('V_CE(sat) vs I_C\n(forced beta = 10)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.set_xlim(0, 210)

# Power dissipated in saturated BJT
P_sat_typ = I_C_sat * V_CE_sat_typ  # mA * V = mW
P_sat_max = I_C_sat * V_CE_sat_max

ax2.plot(I_C_sat, P_sat_typ, 'b-o', linewidth=2, markersize=8, label='Typical')
ax2.plot(I_C_sat, P_sat_max, 'r--s', linewidth=2, markersize=8, label='Maximum')
ax2.axhline(y=625, color='gray', linestyle='--', linewidth=2, label='P_D(max) = 625 mW')

ax2.set_xlabel('Collector Current I_C (mA)', fontsize=12)
ax2.set_ylabel('Power in BJT (mW)', fontsize=12)
ax2.set_title('Power Dissipation When Saturated', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.set_xlim(0, 210)
ax2.set_ylim(0, 200)

plt.tight_layout()
plt.show()

print("Even at the maximum I_C of 200 mA:")
print(f"  Worst-case saturation power = 200 mA * 0.75V = 150 mW")
print(f"  This is well within the 625 mW limit.")
print(f"  Saturated switches are very power-efficient — most power goes to the load.")

---
## Experiment — Design a Switch Circuit Using Only Datasheet Values

**Design task**: Use a 2N3904 to switch a 12V relay coil (R_coil = 240 ohms) from a
5V logic signal. Design using ONLY the worst-case datasheet values.

### Circuit

```
    V_relay (+12V bench supply)
         |
     [Relay coil]
     [240 ohms  ]    1N4001 (flyback diode)
         |            |    |
         +------(K)---D----+
         |                 |
         C                 |
    B ---| 2N3904          |
    |     E                |
   [R_B]  |                |
    |    GND              GND
  V_in
  (5V logic)
```

### Measurements

After building and testing, measure with the Klein MM300:

| Parameter | Calculated | Measured |
|:---|:---:|:---:|
| I_C (relay current) | _____ | _____ |
| V_CE(sat) | _____ | _____ |
| V_BE | _____ | _____ |
| I_B | _____ | _____ |
| P_BJT | _____ | _____ |

In [ ]:
# Design exercise: Calculate component values from datasheet specs

print("=" * 60)
print("DESIGN EXERCISE: 2N3904 Relay Driver")
print("Using worst-case datasheet values")
print("=" * 60)

# Given
V_relay = 12.0     # relay supply voltage
R_coil = 240       # relay coil resistance
V_in = 5.0         # logic input voltage

# From datasheet (worst-case values)
V_CE_sat_max = 0.3   # V, max at ~50 mA (using I_C/I_B = 10)
V_BE_max = 0.95      # V, max V_BE(sat)
hFE_min = 100        # minimum beta at ~50 mA... but wait!
# At 50 mA, datasheet says hFE_min = 60!
hFE_min_at_50mA = 60

print(f"\nStep 1: Calculate relay current")
I_C = (V_relay - V_CE_sat_max) / R_coil
print(f"  I_C = (V_relay - V_CE(sat)) / R_coil")
print(f"       = ({V_relay} - {V_CE_sat_max}) / {R_coil}")
print(f"       = {I_C*1e3:.1f} mA")

print(f"\nStep 2: Check absolute maximum ratings")
print(f"  I_C = {I_C*1e3:.1f} mA < 200 mA max?  YES")
P_BJT = V_CE_sat_max * I_C
print(f"  P_BJT = {V_CE_sat_max}V x {I_C*1e3:.1f}mA = {P_BJT*1e3:.1f} mW < 625 mW max?  YES")
print(f"  V_CE_max in circuit = {V_relay}V < 40V V_CEO?  YES")

print(f"\nStep 3: Calculate base current (using datasheet hFE at actual current)")
print(f"  CAUTION: At I_C ~ {I_C*1e3:.0f} mA, datasheet hFE min = {hFE_min_at_50mA}")
print(f"  (NOT 100-300 — that's at 10 mA!)")
I_B_min = I_C / hFE_min_at_50mA
print(f"  I_B(min) = I_C / hFE(min) = {I_C*1e3:.1f} / {hFE_min_at_50mA} = {I_B_min*1e3:.2f} mA")

forced_beta = 10
I_B_design = I_C / forced_beta
print(f"\n  For reliable saturation (forced beta = {forced_beta}):")
print(f"  I_B(design) = I_C / {forced_beta} = {I_C*1e3:.1f} / {forced_beta} = {I_B_design*1e3:.1f} mA")

print(f"\nStep 4: Calculate R_B")
R_B = (V_in - V_BE_max) / I_B_design
print(f"  R_B = (V_in - V_BE(max)) / I_B")
print(f"       = ({V_in} - {V_BE_max}) / {I_B_design*1e3:.1f} mA")
print(f"       = {R_B:.0f} ohms")

# Standard value
R_B_std = 820  # nearest standard below calculated
print(f"  Use standard value: {R_B_std} ohms (round DOWN for more base current)")

I_B_actual = (V_in - V_BE_max) / R_B_std
print(f"\n  Actual I_B = ({V_in} - {V_BE_max}) / {R_B_std} = {I_B_actual*1e3:.2f} mA")
print(f"  Effective forced beta = I_C / I_B = {I_C / I_B_actual:.1f}")
print(f"  This is {'< hFE_min = ' + str(hFE_min_at_50mA) + ' -- good, will saturate!' if I_C / I_B_actual < hFE_min_at_50mA else 'WARNING: may not saturate!'}")

print(f"\n" + "=" * 60)
print(f"FINAL DESIGN")
print(f"=" * 60)
print(f"  R_B = {R_B_std} ohms")
print(f"  Flyback diode: 1N4001 across relay coil")
print(f"  Expected I_C = {I_C*1e3:.1f} mA")
print(f"  Expected V_CE(sat) < 0.3V")
print(f"  Power in BJT < {V_CE_sat_max * I_C * 1e3:.0f} mW")

In [ ]:
# Interactive design tool: change requirements and see design update

def bjt_switch_designer(V_supply=12, R_load=240, V_logic=5.0):
    """Interactive BJT switch design from datasheet values."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Calculate
    V_CE_sat = 0.3  # worst case
    V_BE = 0.95     # worst case
    I_C = (V_supply - V_CE_sat) / R_load
    
    # Check if within 2N3904 limits
    if I_C > 0.200:
        status = 'OVER CURRENT LIMIT'
        status_color = 'red'
    elif I_C * V_CE_sat > 0.625:
        status = 'OVER POWER LIMIT'
        status_color = 'red'
    else:
        status = 'WITHIN LIMITS'
        status_color = 'green'
    
    # Find beta at this current (interpolate from datasheet)
    if I_C * 1000 < 1:
        hFE_min = 40
    elif I_C * 1000 < 10:
        hFE_min = 70
    elif I_C * 1000 < 50:
        hFE_min = 100
    elif I_C * 1000 < 100:
        hFE_min = 60
    else:
        hFE_min = 30
    
    I_B_design = I_C / 10  # forced beta = 10
    R_B = (V_logic - V_BE) / I_B_design if I_B_design > 0 else 1e6
    
    # Left: SOA with operating point
    V_CE_range = np.linspace(0.01, 45, 500)
    I_C_SOA = np.minimum(0.200, 0.625 / V_CE_range)
    I_C_SOA[V_CE_range > 40] = 0
    
    ax1.loglog(V_CE_range, I_C_SOA * 1e3, 'r-', linewidth=2)
    ax1.fill_between(V_CE_range, 0.01, I_C_SOA * 1e3, alpha=0.1, color='green')
    
    # Plot operating point
    ax1.plot(V_CE_sat, I_C * 1e3, 'o', color=status_color, markersize=15, 
             markeredgecolor='black', markeredgewidth=2, zorder=5,
             label=f'Q-point: {V_CE_sat}V, {I_C*1e3:.1f}mA')
    
    ax1.set_xlabel('V_CE (V)', fontsize=12)
    ax1.set_ylabel('I_C (mA)', fontsize=12)
    ax1.set_title(f'SOA Check: {status}', fontsize=13, fontweight='bold', color=status_color)
    ax1.legend(fontsize=11)
    ax1.set_xlim(0.1, 50)
    ax1.set_ylim(0.1, 300)
    
    # Right: Design summary
    ax2.axis('off')
    
    summary = (
        f"BJT SWITCH DESIGN SUMMARY\n"
        f"{'='*40}\n\n"
        f"REQUIREMENTS:\n"
        f"  V_supply     = {V_supply} V\n"
        f"  R_load       = {R_load} ohms\n"
        f"  V_logic      = {V_logic} V\n\n"
        f"CALCULATED (worst-case):\n"
        f"  I_C          = {I_C*1e3:.1f} mA\n"
        f"  hFE(min)     = {hFE_min} (at this I_C)\n"
        f"  I_B(design)  = {I_B_design*1e3:.2f} mA\n"
        f"  R_B          = {R_B:.0f} ohms\n"
        f"  P_BJT        = {I_C * V_CE_sat * 1e3:.1f} mW\n\n"
        f"STATUS: {status}\n"
        f"  I_C < 200 mA? {'YES' if I_C < 0.200 else 'NO!'}\n"
        f"  P < 625 mW?   {'YES' if I_C * V_CE_sat < 0.625 else 'NO!'}\n"
        f"  V < 40V?      {'YES' if V_supply < 40 else 'NO!'}"
    )
    ax2.text(0.05, 0.95, summary, transform=ax2.transAxes, fontsize=12,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor=status_color, linewidth=2))
    
    plt.tight_layout()
    plt.show()

widgets.interact(
    bjt_switch_designer,
    V_supply=widgets.FloatSlider(min=3, max=36, step=0.5, value=12, description='V_supply (V):'),
    R_load=widgets.IntSlider(min=10, max=1000, step=10, value=240, description='R_load (ohms):'),
    V_logic=widgets.FloatSlider(min=2.5, max=5.0, step=0.1, value=5.0, description='V_logic (V):')
);

---
## Simulation — Build It in Falstad

Build a BJT relay driver to practice the datasheet parameters in context.

**What to build:**

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an NPN transistor
3. Connect a relay coil (modeled as an inductor, ~100 mH, with ~100Ω series resistance) from the 12V supply to the collector
4. Add a flyback diode across the relay coil (cathode toward 12V) — right-click → **Draw** → **Active Components** → **Add Diode**
5. Connect the emitter to ground
6. Connect a switch and 4.7 kΩ base resistor to 5V for the base drive

**What to observe:**
- Toggle the switch: the relay coil energizes (current flows through it)
- Hover over the transistor and check V_CE — it should be low (V_CE(sat) from the datasheet, typically 0.2–0.3V)
- Check V_BE — should be ~0.7V
- Remove the flyback diode, toggle the switch off, and watch the voltage spike across the transistor — this is why the datasheet specifies V_CEO (max collector-emitter voltage)
- Put the diode back — the spike is clamped safely

**What to think about:**
- The base resistor value was chosen to ensure enough base current for saturation: I_B = (5V - 0.7V) / 4.7kΩ ≈ 0.9 mA, which should saturate the transistor for relay currents under ~90 mA (assuming β ≥ 100)
- This is exactly the kind of calculation the datasheet enables

---
## Datasheet Connection — Complete Reference Table

Here is a summary mapping every datasheet section to its practical use:

| Datasheet Section | What You Find | When You Need It |
|:---|:---|:---|
| **Pin Diagram** | Pin order (E-B-C) | ALWAYS — first thing to check |
| **Absolute Maximum: V_CEO** | 40V | Choosing supply voltage, inductive loads |
| **Absolute Maximum: I_C** | 200 mA | Sizing load current |
| **Absolute Maximum: P_D** | 625 mW @ 25C | Power budget, heatsink decisions |
| **h_FE vs I_C table** | 30-300 (varies!) | Calculating R_B for switches |
| **V_CE(sat)** | 0.2-0.3V | Power loss when ON, voltage to load |
| **V_BE(on)** | 0.65-0.95V | Calculating R_B accurately |
| **f_T** | 300 MHz | Amplifier bandwidth planning |
| **Switching times** | ~300 ns total | Max switching frequency |
| **Thermal resistance** | 200 C/W (TO-92) | Thermal derating at high temperatures |
| **SOA graph** | Combined limits | Verifying design is safe |
| **hFE vs IC graph** | Beta curve shape | Choosing operating point for amplifier |

---
## Checkpoint Questions

**Q1.** The 2N3904 datasheet says P_D(max) = 625 mW. Your circuit dissipates 400 mW and
operates in an enclosure where ambient temperature reaches 70 C. Is this safe?

<details><summary>Answer</summary>

NO! At 70C ambient:
P_D(max) = (150 - 70) / 200 = 80 / 200 = 400 mW.

Your 400 mW load is RIGHT at the limit. This gives zero safety margin and the junction
will be at exactly 150C. In practice, you should derate to at most 80% of the limit,
so the maximum safe dissipation is 320 mW. You need to reduce power or improve cooling.
</details>

**Q2.** You are designing a switch for I_C = 80 mA. The datasheet says h_FE min = 60 at
I_C = 50 mA and h_FE min = 30 at I_C = 100 mA. What minimum I_B do you need? What R_B
would you use with a 3.3V input?

<details><summary>Answer</summary>

At 80 mA, interpolate: hFE(min) ~ 40 (conservative estimate between 60 and 30).

For saturation with forced beta = 10: I_B = 80 mA / 10 = 8 mA.

R_B = (3.3V - 0.95V) / 8 mA = 294 ohms. Use 270 ohms (standard value, rounds down).

Note: Using the 0.95V worst-case V_BE from the datasheet, not the 0.7V approximation.
</details>

**Q3.** Why is V_EBO max only 6V when V_CEO max is 40V? What design situation could
violate V_EBO?

<details><summary>Answer</summary>

V_EBO is low because the emitter-base junction has heavy doping, creating a narrow
depletion region that breaks down at low voltage. V_CEO has a lightly-doped collector
creating a wide depletion region with higher breakdown.

V_EBO can be violated when:
- A negative turn-off voltage is applied to the base (speed-up technique)
- The base is connected to a signal that swings negative
- A voltage divider biasing network leaves a large negative voltage on the base
  when the input swings low
</details>

**Q4.** The datasheet shows h_FE = 100-300 at I_C = 10 mA, but only 30 min at I_C = 100 mA.
What physical mechanism causes beta to drop at high currents?

<details><summary>Answer</summary>

At high collector currents, the base region becomes flooded with carriers (high-level
injection). The effective base width increases and recombination in the base increases,
reducing the fraction of emitter-injected carriers that reach the collector. This is
called the "Kirk effect" or "base push-out." Additionally, ohmic voltage drops in
the base and emitter bulk regions reduce the effective junction bias.
</details>

**Q5.** You find a BJT in your parts bin labeled "2N3904" but are not sure if it is real.
Using only your Klein MM300 in diode-test mode, describe the tests you would perform to
verify it is a functional NPN transistor and identify its pins.

<details><summary>Answer</summary>

1. Find the BASE: Test all six combinations of two pins. The base is the pin that shows
   a forward diode drop (~0.6-0.7V) to BOTH other pins in one polarity (red lead on base
   for NPN, black lead on base for PNP).

2. Confirm NPN: With the base identified, red lead on base should show ~0.6-0.7V to
   both collector and emitter. Black lead on base should show OL to both.

3. Distinguish E from C: The BE junction typically shows a slightly lower forward voltage
   than the BC junction (because the emitter is more heavily doped). Or: with red on base
   and black on the suspected emitter, the reading is typically 0.01-0.02V lower than
   base-to-collector.

4. Verify no shorts: Collector-to-emitter should read OL in both directions (with no
   base connection).

5. Final check: Build a simple switch circuit and verify it switches. The pinout diagram
   in the datasheet is the definitive reference.
</details>